# Generate index components using dedicated service - F#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Index sample is designed to help you searching and requesting index components reference data by using a wide range of commonly used fields.

It takes an index code as per input filter and returns the *entries* matching the request.

### Services used
This sample uses *gRPC requests* in order to retrieve index components reference data from the hosted service. The queried endpoint in this script are:
* *StaticDataService*: to directly retrieve reference data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run Index components sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.33.*-pre*"
#r "nuget: CsvHelper"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.StaticData.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
// Import/export csv files
open CsvHelper
open System.IO
open System.Globalization

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Request creation

The following code snippet enables to select the **index** by its *Name/code*:

In [ ]:
// Set index and exchange codes
let index = "Nasdaq 100"
let exchange = "XNGS"

The following code snippets call the service, generate the request and return the reply: 

In [ ]:
// Instantiate the service
let service = new StaticDataService.StaticDataServiceClient(channel)

let request = new StaticDataRequest(AssetType = AssetType.Equity, Index = index,  Count = 1000 )// By default the count is set to 100, make sure to retrieve all the components

// Call the service
let reply = service.StaticData(request, headers)

// Display the results
display(reply.Equities.Count)

### Step 4: Retrieve data

#### 4.1 Retrieve instruments' identifiers (mapping)

In [ ]:
// Create a class to handle the instrument's mapping codes
type Instrument =
    {
        Ticker: string
        Exchange: string
        Name: string
        Isin: string
        Cusip: string
        Sedol: string
        Bloomberg: string
    }

let instruments =
    reply.Equities
    |> Seq.filter (fun e -> e.Identifier.Exchange = exchange )
    |> Seq.map(fun e -> { Ticker = e.Identifier.Ticker; Exchange = e.Identifier.Exchange; Name = e.Name; Isin = e.Isin; Cusip = e.Cusip; Sedol = e.Sedol; Bloomberg = if e.Mapping.ContainsKey("Bloomberg") then e.Mapping.["Bloomberg"] else String.Empty })

In [ ]:
display(instruments)

The following code snippet exports data to *csv* file:

In [ ]:
let d = new DirectoryInfo("output") 
d.Create()
let write instruments =
    let writeRecords (csv : CsvWriter) i =
        csv.WriteRecords(i)
    let write' (writer : StreamWriter )  i =
        using (new CsvWriter(writer, CultureInfo.InvariantCulture)) (fun csv -> writeRecords csv i )
    using (new StreamWriter($"output/{index}.csv")) (fun writer -> write' writer instruments)

instruments
|> write